In [59]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import yfinance as yf
from pathlib import Path

In [60]:
df = pd.read_csv('data.csv')
df.head()
len(df)

50560

In [61]:
#company_tickers = Path('all_symbols.txt').read_text()
#company_tickers = " ".join(line.strip() for line in company_tickers.splitlines())
#data = yf.download(company_tickers, start="2015-12-31", end="2016-01-01", threads = False)

Start preprocessing data.


In [62]:
#fill null expenses with 0
df['xrd'] = df['xad'].fillna(0)
df['xad'] = df['xad'].fillna(0)
df['xsga'] = df['xsga'].fillna(0)
df['xint'] = df['xsga'].fillna(0)

#combine all expenses columns into one column
expenses_list = ['xrd', 'xad', 'xsga', 'xint', 'nopio', 'cogs']
df['expenses'] = df[expenses_list].sum(axis=1)

df = df.drop(columns=expenses_list)

#delete duplicate rows due to formatting
condition = df['indfmt'] != "FS"
df = df[condition]

#drop unneccessary columns
df = df.drop(columns=['indfmt', 'consol', 'popsrc', 'datafmt', 'curcd', 'gvkey', 'costat', 'dlcch'])

#remove rows that are almost completely empty
condition = df.isnull().sum(axis=1) < 5
df = df[condition]

df = df.dropna(subset=['mkvalt'], how='all')

In [70]:
#create column 'future mkvalt' that holds the market value of the next year
condition1 = df['tic'].shift(-1) == df['tic']
condition2 = df['fyear'].shift(-1) == df['fyear'] + 1
df['future_mkvalt'] = np.where(condition1 & condition2, df['mkvalt'].shift(-1), np.nan)

#drop nulls
df = df.dropna(subset=['future_mkvalt'], how='all')

#percent change is the percent change in market value from one year to the next
df['percent_change'] = (df['future_mkvalt']-df['mkvalt'])/df['mkvalt']

#if percent change is greater than 20%, classify as undervalued
df['undervalued'] = (df['percent_change'] > 0.2)

In [71]:
df.head(20)

,tic,fyear,act,ap,at,ceq,che,dlc,dltt,dp,...,sale,spi,txp,txt,xido,mkvalt,expenses,future_mkvalt,percent_change,undervalued
0,AIR,2009.0,863.429,114.906,1501.042,746.906,79.370,100.833,336.191,38.930,...,1352.151,-4.302,3.263,20.986,0.000,777.8348,1368.672,1049.8206,0.349670,True
1,AIR,2010.0,913.985,185.096,1703.727,835.845,57.433,114.075,329.802,59.296,...,1775.782,-1.536,0.000,35.364,-3.313,1049.8206,1754.213,485.2897,-0.537740,False
2,AIR,2011.0,1063.272,201.405,2195.653,864.649,67.720,122.865,669.489,80.333,...,2074.498,-13.864,0.000,25.480,0.000,485.2897,2042.744,790.0029,0.627900,True
3,AIR,2012.0,1033.700,149.300,2136.900,918.600,75.300,86.400,622.200,108.600,...,2167.100,-21.100,0.000,26.700,0.000,790.0029,2136.100,961.3080,0.216841,True
4,AIR,2013.0,1116.900,171.100,2199.500,999.500,89.200,69.700,564.300,113.400,...,2035.000,0.000,0.000,32.100,0.000,961.3080,1979.700,1046.3954,0.088512,False
5,AIR,2014.0,954.100,142.300,1515.000,845.100,54.700,69.000,85.000,92.300,...,1594.300,-48.400,0.000,-28.500,64.700,1046.3954,1678.700,842.5112,-0.194844,False
6,AIR,2015.0,873.100,163.400,1442.100,865.800,31.200,12.000,136.100,70.800,...,1662.600,-0.400,1.100,18.800,7.200,842.5112,1696.200,1200.3288,0.424704,True
10,AAL,2010.0,6838.000,1156.000,25088.000,-3945.000,4946.000,1883.000,9253.000,995.000,...,22170.000,NaN,0.000,-35.000,0.000,2597.5755,23909.000,117.3438,-0.954826,False
11,AAL,2011.0,6757.000,1007.000,23848.000,-7111.000,4739.000,1518.000,6702.000,981.000,...,24022.000,-886.000,0.000,0.000,0.000,117.3438,26599.000,266.5571,1.271591,True
12,AAL,2012.0,7072.000,1244.000,23510.000,-7987.000,4742.000,1419.000,7116.000,940.000,...,24855.000,-2595.000,0.000,-569.000,0.000,266.5571,26911.000,6591.9923,23.730132,True


In [72]:
df.isnull().sum()

tic                  0
fyear                0
act               3477
ap                  49
at                   0
ceq                 17
che                  0
dlc                  2
dltt                83
dp                 521
dvc                 27
ib                   0
intan              263
invt               189
ivao               836
lct               3429
lt                  37
ppent              285
rect                85
sale                 0
spi                187
txp               2648
txt                  2
xido                 0
mkvalt               0
expenses             0
future_mkvalt        0
percent_change       0
undervalued          0
dtype: int64

In [66]:
df.groupby('fyear').agg({'mkvalt': lambda x: x.isnull().sum()})

,mkvalt
fyear,
2009.0,0
2010.0,0
2011.0,0
2012.0,0
2013.0,0
2014.0,0
2015.0,0
2016.0,0
2017.0,0


In [67]:
len(df)

26183